# Trusted AI IBM-Azure usecase: monitoring Azure model

**Build & Deploy** Machine Learning model in **Azure Machine Learning Studio**

**Monitor** the model in **IBM Watson OpenScale**

### Contents

1. [Set up the environment](#setup_environment)
1. [Explore and prepare training data](#explore_prepare_data)
1. [Create train and test dataset](#train_test_set)
1. [Train the model](#train_model)
1. [Save the model in Azure](#save_model)
1. [Create a custom entry script](#custom_score_script)
1. [Create an online endpoint](#create_endpoint)
1. [Create a custom environment for python](#create_custom_environment)
1. [Deploy and score in Azure](#deploy_model)
1. [Configure IBM OpenScale](#configure_openscale)

In [ ]:
%pip install ibm_cloud_sdk_core
%pip install ibm-watson-openscale

In [ ]:
import os

import pandas as pd
import numpy as np

<a id="setup_environment"></a>
## 1. Set up the environment

**IBM credentials**

To authenticate to IBM Watson OpenScale in the IBM Cloud, you need api_key and service location.

Using [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) or directly through the IBM Cloud portal.

Using IBM Cloud CLI:

```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

NOTE: To get the Service URL [Endpoint URLs section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).

**Action**: Enter your api_key and location in the following cell.

In [ ]:
IBM_API_KEY = 'API_KEY'

WOS_DB_CREDENTIALS=None
WOS_SCHEMA_NAME = 'azure_dm'

IBM_IAM_URL="https://iam.ng.bluemix.net/oidc/token"

Azure credentials

In [ ]:
AZURE_ENGINE_CREDENTIALS =  {
    "client_id": 'CLIENT_ID',
    "client_secret": 'CLIENT_SECRET',
    "tenant": 'TENANT',
    "subscription_id": 'SUBSCRIPTION_ID'
}

<a id="explore_prepare_data"></a>
## 2. Explore and prepare training data

NOTE: read from `/data` directory if running locally

In [ ]:
df = pd.read_csv('../data/credit_risk_training.csv')
df.head()

In [ ]:
print('Columns: ', list(df.columns))
print('Number of columns: ', len(df.columns))

<a id="train_test_set"></a>
## 3. Create train and test dataset

NOTE: Test dataset (20%) and Training dataset (80%)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2)

In [ ]:
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In [ ]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

<a id="train_model"></a>
## 4. Train the model

Create a Scikit-learn models.

In [ ]:
MODEL_NAME = "Scikit Credit Risk Model Azure ML"
MODEL_NAME_SHORT = 'azure_credit_risk_model'
DEPLOYMENT_NAME = "Scikit Credit Risk Deployment Azure ML"

In [ ]:
risk_model = pipeline.fit(train_data.drop('Risk', axis=1), train_data.Risk)

In [ ]:
from sklearn.metrics import roc_auc_score

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)

print(auc)

In [ ]:
import joblib

joblib.dump(risk_model, '../model/' + MODEL_NAME_SHORT+ ".pkl")

<a id="save_model"></a>
## 5. Save the model in Azure

In [ ]:
%pip install azure-ai-ml azure-identity

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [ ]:
os.environ["AZURE_CLIENT_ID"] = AZURE_ENGINE_CREDENTIALS["client_id"]
os.environ["AZURE_TENANT_ID"] = AZURE_ENGINE_CREDENTIALS["tenant"]
os.environ["AZURE_CLIENT_SECRET"] = AZURE_ENGINE_CREDENTIALS["client_secret"]

In [ ]:
# Authenticate
# https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?view=azureml-api-2&tabs=sdk

credential = DefaultAzureCredential()

# credential.get_token("https://management.azure.com/.default")

# Get a handle to the workspace
az_ml_client = MLClient(
    credential=credential,
    subscription_id=AZURE_ENGINE_CREDENTIALS["subscription_id"],
    resource_group_name="watsonx_governance",
    workspace_name="ml_integration",
)

In [ ]:
ws = az_ml_client.workspaces.get('ml_integration')
print("LOCATION:", ws.location, "NAME:", ws.name, "RESOURCE_GROUP:", ws.resource_group)

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path='../model/'+MODEL_NAME_SHORT+'.pkl',
    type=AssetTypes.CUSTOM_MODEL,
    name=MODEL_NAME_SHORT,
    description='Random Forest Model to classify credit risk with probability',

)

az_ml_client.models.create_or_update(file_model)

print('Name:', file_model.name)
print('Version:', file_model.version)

In [ ]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in az_ml_client.models.list(name=MODEL_NAME_SHORT)]
)
print(f'Latest model is version "{latest_model_version}" ')

<a id="custom_score_script"></a>
## 6. Create a custom entry script for the scoring response

In [ ]:
%%writefile ../script/azure_score.py
import os
import json
import joblib

import pandas as pd

def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    model_path = os.path.join(str(os.getenv("AZUREML_MODEL_DIR")), "azure_credit_risk_model.pkl") 
    model = joblib.load(model_path)

    print("===> Init complete")

def run(input_payload):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    print("===> Request received")
    try:
        if type(input_payload) is str:
            dict_data = json.loads(input_payload)
        else:
            dict_data = input_payload

        data = pd.DataFrame.from_dict(dict_data["input"])
        predictions = model.predict(data)
        scores = model.predict_proba(data)
        risk_column = []
        proba_column = []
        proba_vector = []

        for pred, proba in zip(predictions, scores):
            risk_column.append(pred)
            proba_vector.append([proba[0], proba[1]])
            if pred == "No Risk":
                proba_column.append(proba[0])
            else:
                proba_column.append(proba[1])
        data["Scored Labels"] = risk_column
        data["Scored Probabilities"] = proba_column
        data["ProbabilityVector"] = proba_vector

        result = { "output": data.to_dict('records') }
        print("===> Request processed")

        return result
    except Exception as e:
        result = str(e)
        return { "error": result }


<a id="create_endpoint"></a>
## 7. Create an online endpoint

In [ ]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = MODEL_NAME_SHORT + '_' + str(uuid.uuid4())[:8]
online_endpoint_name = online_endpoint_name.replace('_', '-')

print(online_endpoint_name)

In [ ]:
# Expect the endpoint creation to take a few minutes

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name, 
    description="This is a sample online endpoint",
    auth_mode="key"
)

endpoint = az_ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

In [ ]:
endpoint = az_ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

<a id="create_custom_environment"></a>
## 8. Create a custom environment for python

In [ ]:
environment = Environment(
    name="azure-sklearn1-3-env",
    description="Environment created from a Docker image plus Conda environment",
    conda_file="../az-environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [ ]:
az_ml_client.environments.create_or_update(environment)

In [ ]:
# Let's pick the latest version of the environment
latest_env_version = max(
    [int(m.version) for m in az_ml_client.environments.list(name="azure-sklearn1-3-env")]
)
print(f'Latest model is version "{latest_env_version}" ')

In [ ]:
environment = az_ml_client.environments.get(name="azure-sklearn1-3-env", version=latest_env_version)

<a id="deploy_model"></a>
## 9. Deploy and score in Azure

NOTE: Deploy and score the model deployed at Azure ML Studio

In [ ]:
%%time
model = az_ml_client.models.get(name=MODEL_NAME_SHORT, version=latest_model_version)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_D2as_v4", # Standard_DS3_v2
    instance_count=1,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="../script", scoring_script="azure_score.py"
    ),
)

blue_deployment = az_ml_client.begin_create_or_update(blue_deployment).result()

In [ ]:
# Test the deployment with some sample data
import json

with open('./azure_credit_risk_input_data.json', 'w') as f:
    json.dump({'input': train_data.drop('Risk', axis=1)[-1:].to_dict('records')}, f)

In [ ]:
output = az_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="./azure_credit_risk_input_data.json",
)

print(output)

Delete `azure_credit_risk_input_data.json` file from filesystem

In [ ]:
os.remove('./azure_credit_risk_input_data.json')

<a id="configure_openscale"></a>
## 10. Configure IBM OpenScale

https://client-docs.aiopenscale.cloud.ibm.com/html/index.html#

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(
        apikey=IBM_API_KEY
    )

wos_client = APIClient(authenticator=authenticator)

print('Watson OpenScale on IBM Cloud!')
print(wos_client.version)

### 10.1 DataMart setup

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts

if len(data_marts) == 0:
    if WOS_DB_CREDENTIALS is not None:
        if WOS_SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=WOS_DB_CREDENTIALS['hostname'],
                        username=WOS_DB_CREDENTIALS['username'],
                        password=WOS_DB_CREDENTIALS['password'],
                        db=WOS_DB_CREDENTIALS['database'],
                        port=WOS_DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=WOS_DB_CREDENTIALS['sslmode'],
                        certificate_base64=WOS_DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= WOS_SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

### 10.2 Add service provider Azure Machine Learning Service

In [ ]:
SERVICE_PROVIDER_NAME = "Azure Machine Learning Service"
SERVICE_PROVIDER_DESCRIPTION = "Added by Azure Machine Learning Studio notebook."

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.AZURE_MACHINE_LEARNING,
        credentials=AzureCredentials(
            subscription_id= AZURE_ENGINE_CREDENTIALS['subscription_id'], 
            client_id = AZURE_ENGINE_CREDENTIALS['client_id'], 
            client_secret= AZURE_ENGINE_CREDENTIALS['client_secret'],
            tenant = AZURE_ENGINE_CREDENTIALS['tenant']
        ),
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

In [ ]:
wos_client.service_providers.show()

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
print(asset_deployment_details)